## Mount the Drive, and Change to Google Drive Folder

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

%cd /content/drive/MyDrive/MSc.-Dissertations/1/Files
%ls

Mounted at /content/drive
/content/drive/MyDrive/MSc.-Dissertations/1/Files
classification.ipynb  properties.csv      Robin.ipynb
flist.txt             randomsample.ipynb  street_view/


## Import Libraries

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import random
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

## Count the Number of Files, and Take Random Samples from the Image Files

In [3]:
# !ls street_view
# count how many files and write the filenames into a file
!ls street_view -1 | wc -l 
!ls street_view/*.jpg > flist.txt
flist = list(pd.read_csv('flist.txt', header = None)[0])

# Set seed so sample is reproducible 
random.seed(99)  # set this to an integer value!!!
nsamp = 10
flist_sub = random.sample(flist, nsamp)
flist = flist_sub

print(flist)

16966
['street_view/gsv_c6fba904-e168-4fe7-8851-04d409f3db12.jpg', 'street_view/gsv_bb9597b6-655e-448e-b6a6-79fbe1c26c43.jpg', 'street_view/gsv_621bbc77-62df-40fd-af32-09c779be9958.jpg', 'street_view/gsv_57c8b3a2-33bb-4c67-b1b5-801b644ab851.jpg', 'street_view/gsv_70fb5033-604f-421e-8afc-4ffc4cf0206b.jpg', 'street_view/gsv_7a067fe3-f764-407e-a6b4-99de96eb6743.jpg', 'street_view/gsv_42118ab6-ee09-4c0a-b241-d05807afa745.jpg', 'street_view/gsv_2afe400f-f2e3-4b1a-88fe-5567c7dc84d7.jpg', 'street_view/gsv_7b947570-ae42-418b-a43f-f571b120d93c.jpg', 'street_view/gsv_bcbdc03c-25e2-4fb7-9af7-5bd030592b45.jpg']


## Overview of the `properties` Dataset

In [4]:
properties = pd.read_csv('properties.csv')
properties.head()

,Unnamed: 0,address,propertyType,bedrooms,detailUrl,location_lat,location_lng,property_id
0,0,"12, Gorsey Brigg, Dronfield Woodhouse, Dronfie...",Terraced,3.0,https://www.rightmove.co.uk/house-prices/detai...,53.29986,-1.49446,60d9dd15-c5a0-4d9c-a341-a1d47add49d5
1,0,"5, Highgate Lane, Dronfield, Derbyshire S18 1UB",Detached,4.0,https://www.rightmove.co.uk/house-prices/detai...,53.29135,-1.45975,4a586e80-181a-4b82-b5c3-2d789436bb14
2,0,"125, Gosforth Lane, Dronfield, Derbyshire S18 1RB",Detached,3.0,https://www.rightmove.co.uk/house-prices/detai...,53.29763,-1.47573,93680b6c-237e-44d3-8f40-959a14b80cad
3,0,"80, Shakespeare Crescent, Dronfield, Derbyshir...",Detached,3.0,https://www.rightmove.co.uk/house-prices/detai...,53.29259,-1.45644,5d49758b-f148-4d06-bbae-3eb23f5c68fb
4,0,"21, Gainsborough Road, Dronfield, Derbyshire S...",Detached,NaN,https://www.rightmove.co.uk/house-prices/detai...,53.29740,-1.48503,4645f5eb-de7c-474f-8d7e-b59fa8c55f19


In [5]:
properties.propertyType = properties.propertyType.astype('category')
properties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17550 entries, 0 to 17549
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Unnamed: 0    17550 non-null  int64   
 1   address       17550 non-null  object  
 2   propertyType  17550 non-null  category
 3   bedrooms      11505 non-null  float64 
 4   detailUrl     17550 non-null  object  
 5   location_lat  17550 non-null  float64 
 6   location_lng  17550 non-null  float64 
 7   property_id   17550 non-null  object  
dtypes: category(1), float64(3), int64(1), object(3)
memory usage: 977.2+ KB


In [6]:
properties.describe()

,Unnamed: 0,bedrooms,location_lat,location_lng
count,17550.0,11505.000000,17550.000000,17550.000000
mean,0.0,2.871186,52.912264,-2.330492
std,0.0,1.010339,1.833830,1.262468
min,0.0,0.000000,50.617080,-4.268950
25%,0.0,2.000000,51.232830,-3.067290
50%,0.0,3.000000,53.095885,-2.658955
75%,0.0,3.000000,53.846760,-1.712750
max,0.0,6.000000,55.910540,0.719990


In [7]:
properties.propertyType.value_counts()

Detached         4134
Semi-Detached    4056
Unknown          3900
Terraced         3666
Flat             1794
Name: propertyType, dtype: int64

## A Subsample of the `properties` Dataset

In [8]:
flist_id = list(map(lambda string: string[16 : -4], flist))
properties_sub = properties.loc[properties['property_id'].isin(flist_id)]
properties_sub.head()

,Unnamed: 0,address,propertyType,bedrooms,detailUrl,location_lat,location_lng,property_id
2550,0,"64, Queens Crescent, Livingston, West Lothian ...",Unknown,1.0,https://www.rightmove.co.uk/house-prices/detai...,55.89959,-3.54215,7b947570-ae42-418b-a43f-f571b120d93c
4740,0,"11, Meadow Drive, Aughton, Ormskirk, Lancashir...",Detached,4.0,https://www.rightmove.co.uk/house-prices/detai...,53.55281,-2.90198,bcbdc03c-25e2-4fb7-9af7-5bd030592b45
5282,0,"13, Queens Crescent, Stoke-sub-hamdon, Somerse...",Semi-Detached,2.0,https://www.rightmove.co.uk/house-prices/detai...,50.95956,-2.74961,621bbc77-62df-40fd-af32-09c779be9958
9038,0,"1, Beckenham Terrace, North Street, Westbourne...",Terraced,NaN,https://www.rightmove.co.uk/house-prices/detai...,50.86440,-0.92514,2afe400f-f2e3-4b1a-88fe-5567c7dc84d7
9040,0,"24, Kelsey Avenue, Southbourne, Emsworth, West...",Semi-Detached,3.0,https://www.rightmove.co.uk/house-prices/detai...,50.85245,-0.90680,bb9597b6-655e-448e-b6a6-79fbe1c26c43


## PCA

In [9]:
Img_mat = np.asarray(list(map(lambda x: np.asarray(Image.open(x)).reshape(1, 1, -1), flist))).reshape(len(flist), -1)
S_Img_mat = StandardScaler().fit_transform(Img_mat)
S_Img_mat[0 : 5, :]

array([[ 0.52275224,  0.36726415,  0.03478697, ..., -0.44868108,
        -0.3040828 ,  0.02774568],
       [ 1.06087954,  1.15671044,  0.92027342, ...,  0.32048648,
         0.48917669,  0.86011618],
       [-1.51444398, -1.55486597, -0.62932787, ..., -0.19229189,
        -0.96513238, -0.94335323],
       [ 0.06150026,  0.64185416,  1.11002051, ...,  2.24340538,
         2.20790558,  1.8312151 ],
       [-0.20756339, -1.00568594, -2.17892915, ...,  0.57687567,
         0.88580643,  0.86011618]])

In [10]:
pca = PCA(svd_solver = 'arpack')
pc_Img_mat = pca.fit_transform(S_Img_mat)
pc_Img_mat[0 : 5, :]

array([[-167.54410011,   71.44112742,   78.74720962, -181.17064817,
         -47.71883847,  312.11660217, -196.35744986, -162.36553159,
         -43.68932833],
       [ -96.66431258,   66.39292822,  -89.62138961,  359.52157222,
        -159.96282428,   17.43097558,  185.14871963, -214.14445709,
         -43.88881301],
       [ 304.71104867, -210.46395101, -362.08047967, -102.56106577,
        -146.7655864 ,  -19.89204132,  -70.40876784,  -12.26911483,
         176.93975204],
       [ 219.41473717,  609.76969159,  -63.77046019,  -77.5217922 ,
          44.54396611,  -68.66293564,   15.79904641,   62.38354105,
          17.14052005],
       [ 513.35477567, -144.06548414,  404.16992558,   -3.80077434,
         -12.40995733, -109.47599533,  -21.286648  ,  -91.18029398,
         -12.91014398]])